<a href="https://colab.research.google.com/github/diqnfl777/2022F-Ajou-ML-TEAM3/blob/main/CrawlTest_with_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests

cookies = {
    'JSESSIONID': 'E41133919B0B2AA40FA1AEA15743A96A.pyxis-api1',
    'apt.uid': 'AP-PQQY5YJEHTTA-2-1645597751454-98837791.0.2.76c966dc-315c-45f5-9b34-1bd88670682a',
    '_ga': 'GA1.3.1573260349.1630468655',
    '_ga_B05H7FL8XH': 'GS1.1.1665378882.8.0.1665379021.0.0.0',
    '_ga_1WWMRKJP34': 'GS1.1.1665378882.8.1.1665379021.0.0.0',
    '_gid': 'GA1.3.17273452.1667387539',
    'JSESSIONID': 'j4uDYzoUVCSuGyRSYxn051m77D7apDyNb5CAXkJEvHGDP9zxbXdVhS3AsDWbMoLV.yonam_servlet_IPSI01',
    'apt.sid': 'AP-PQQY5YJEHTTA-2-1667460835868-82871188',
    'NG_TRANSLATE_LANG_KEY': '%22ko%22',
    'PYXIS2_UUID': '%22863475d1-b6f4-4154-8753-cca9c6c398c1%22',
    '_gat': '1',
}

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ko',
    'Connection': 'keep-alive',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'JSESSIONID=E41133919B0B2AA40FA1AEA15743A96A.pyxis-api1; apt.uid=AP-PQQY5YJEHTTA-2-1645597751454-98837791.0.2.76c966dc-315c-45f5-9b34-1bd88670682a; _ga=GA1.3.1573260349.1630468655; _ga_B05H7FL8XH=GS1.1.1665378882.8.0.1665379021.0.0.0; _ga_1WWMRKJP34=GS1.1.1665378882.8.1.1665379021.0.0.0; _gid=GA1.3.17273452.1667387539; JSESSIONID=j4uDYzoUVCSuGyRSYxn051m77D7apDyNb5CAXkJEvHGDP9zxbXdVhS3AsDWbMoLV.yonam_servlet_IPSI01; apt.sid=AP-PQQY5YJEHTTA-2-1667460835868-82871188; NG_TRANSLATE_LANG_KEY=%22ko%22; PYXIS2_UUID=%22863475d1-b6f4-4154-8753-cca9c6c398c1%22; _gat=1',
    'Referer': 'https://library.ajou.ac.kr/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'branch': '1',
    'material_type': '1',
    'max': '100',
    'offset': 0,
}

response = requests.get('https://library.ajou.ac.kr/pyxis-api/1/collections/1/search', params=params, cookies=cookies, headers=headers)

In [ ]:
import re
isbn = '9791185992280'
isbn.split('.')[0]
response = requests.get('https://library.ajou.ac.kr/pyxis-api/mashup-aladin/'+ isbn + '/toc')
data = response.json()
if(data['code'] == 'success.retrieved'):
  index_text = data['data']
  index_text = re.sub("[\<].*?[\>]", "", index_text)
  print(index_text)

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

ls: cannot access 'gdrive': Transport endpoint is not connected
gdrive/  sample_data/


In [17]:
import json
import csv
import re
import requests
from multiprocessing import Pool

aladin_headers = {
    "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
    "Connection": "close"}
header = ['제목','저자', '출판', '청구기호','목차']

i_range = 1000
i_start = 0

def WriteCSV(start_i: int):
  i = start_i
  count = start_i
  path = "/content/drive/MyDrive/LibraryCsv/"
  filename = "librarydata_windex_{start_i}-{max_i}.csv".format(start_i = start_i, max_i = start_i+i_range)

  with open(path + filename, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    while i < (start_i + i_range): #454545
      params['offset'] = i
      print("Progress:",(i - start_i)/i_range)
      i+=100
      response = requests.get('https://library.ajou.ac.kr/pyxis-api/1/collections/1/search', params=params, cookies=cookies, headers=headers)
      data = response.json()
      try:
        new_dict = data['data']
      except KeyError:
        print("Error at :",i-100)
        return
      for j in new_dict['list']:
        count+=1
        try:
          branch_dict = j['branchVolumes'][0]
        except IndexError:
          print(j['titleStatement']) #청구기호 조회 안된건 안넣고 제목만 참고용으로 출력
          continue
        isbn = j['isbn']
        if(isbn == None):
          continue;
        isbn.split('.')[0]
        response = requests.get('https://library.ajou.ac.kr/pyxis-api/mashup-aladin/'+ isbn + '/toc', headers=aladin_headers)
        data = response.json()
        if(data['code'] == 'success.retrieved'):
          index_text = data['data']
          index_text = re.sub("[\<].*?[\>]", "", index_text)
          index_text = index_text.replace("&nbsp;","")
          writer.writerow([j['titleStatement'], j['author'],j['publication'], branch_dict['volume'], index_text])
        #이후 https://library.ajou.ac.kr/pyxis-api/mashup-aladin/ * ISBN  * /toc 을 통해 목차까지 같이 조회해서 하는거 진행해야함

if __name__ == "__main__":
  input = []
  process_num = 8
  for i in range(process_num):
    input.append(i_start +i*i_range) 
  print(input)
  with Pool(processes=process_num) as pool:  
    pool.map(WriteCSV, input) 
  


[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000]
Progress: Progress:0.0 
0.0Progress:
 0.0Progress:
 0.0Progress:
 0.0Progress:
 0.0Progress:
 0.0Progress:
 0.0
Progress: 0.1
Progress: 0.1
Progress: 0.1
Progress: 0.1Progress:
 0.1
Progress: 0.1
Progress: 0.1
Progress: 0.1
Progress: 0.2
Progress: 0.2
Progress: 0.2
Progress: 0.2
Progress: 0.2
Progress: Progress:0.2 
0.2
Progress: 0.2
woza afrika
Progress: 0.3
Progress: 0.3
Progress: 0.3
Progress: 0.3
Progress: 0.3
Progress: 0.3
Progress: 0.3
Progress: 0.3
Progress: 0.4
Progress: 0.4
Progress: 0.4
Progress: 0.4
Progress: 0.4
Progress: 0.4
Progress: 0.4
Progress: 0.4
Quantum Chemistry
Progress: 0.5
Progress: 0.5
Progress: 0.5
Progress: 0.5
Progress: 0.5
Progress: 0.5
Progress: 0.5
Progress: 0.5
Convergence :an architectural agenda for energy
Progress: 0.6
Progress: 0.6
Progress: 0.6
Progress: 0.6
Progress: 0.6
Progress: 0.6
Progress: 0.6
Progress: 0.6
Progress: 0.7
Progress: 0.7
Progress: 0.7
Progress: 0.7
Progress: 0.7
Progress: 0.7
Progress